In [ ]:
# ETL project Bootcamp
# Estefanía González
# Paolo Vega
# 20-may-2020 Version 1.0.0

In [1]:
# Modules needed
import pandas as pd
import json
from pandas.io.json import json_normalize
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import os

In [2]:
# Read JSON file
file = 'Resources/DataSources/json/CA_category_id.json'
# Opemn JSON File
data = json.load(open(file))
# Normalize 'items' element inside json tree (where our information is)
items_df = json_normalize(data['items'])
# Remove unwanted columns
items_df = items_df[["id","snippet.title"]].copy()
items_df.head()

,id,snippet.title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [3]:
# Rename columns
items_df = items_df.rename(columns={"id":"CategoryID","snippet.title":"CategoryName"})
# Set index to ID column
#items_df = items_df.set_index("CategoryID")
items_df.head()

,CategoryID,CategoryName
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports


In [4]:
# import variables from config file
from credentials import host
from credentials import pwd
from credentials import usr
from credentials import dialect
from credentials import port
from credentials import db

engine = create_engine(f'{dialect}://{usr}:{pwd}@{host}:{port}/{db}')

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# display tables/classes
display(Base.classes.keys())

# Save references to each table
Category = Base.classes.Category
Country = Base.classes.Country
Video = Base.classes.Video

# Start session
session = Session(engine)

['Category', 'Video', 'Country']

In [20]:
# Read Tables to determine what task to perform in each
category_data = pd.read_sql_table("Category",engine)
country_data = pd.read_sql_table("Country",engine)
video_data = pd.read_sql_table("Video",engine)
engine.dispose()

In [19]:
# insert strategy (empty table)
items_df.head()
if len(category_data) == 0:
    print("Table empty. \n Inserting values...")
    for index, row in items_df.iterrows():
        # remove leading spaces and capitalize the string
        categoryNameTemp = row["CategoryName"].strip().capitalize()
        # create a new object (category)
        newCategory = Category(CategoryID = row["CategoryID"], CategoryName = (categoryNameTemp))
        # insert into DB
        session.add(newCategory)
    print("Done!")
# update Strategy (table with existing rows)
else:
    print("Table not empty")
    # TO DO

# Commit changes
session.commit()

Table empty. 
 Inserting values...
Done!


In [22]:
# Validate data into DB
category_data = pd.read_sql_table("Category",engine)
category_data

,CategoryID,CategoryName
0,1,Film & animation
1,2,Autos & vehicles
2,10,Music
3,15,Pets & animals
4,17,Sports
5,18,Short movies
6,19,Travel & events
7,20,Gaming
8,21,Videoblogging
9,22,People & blogs
